In [72]:
import pandas as pd
from datetime import datetime
from sklearn.neighbors import KernelDensity
import numpy as np
import matplotlib.pyplot as plt

In [73]:
# Leer el archivo CSV
df = pd.read_csv('skateboard.csv')

# Convertir la columna 'date' a datetime especificando el formato
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

# Filtrar por lugar 'Park' y fechas anteriores a mayo de 2024
df_filtrado = df[(df['place'] == 'Park') & (df['date'] < '2024-05-01')]

# Crear diccionarios para hombres y mujeres
mens = {}
womens = {}

# Función para agregar datos al diccionario
def agregar_a_diccionario(diccionario, fila):
    nombre_completo = f"{fila['first_name'].lower()} {fila['last_name'].lower()}"
    if nombre_completo not in diccionario:
        diccionario[nombre_completo] = []
    diccionario[nombre_completo].append([fila['total']])

# Separar los datos por sexo y llenar los diccionarios
for index, row in df_filtrado.iterrows():
    if row['sex'] == 'Mens':
        agregar_a_diccionario(mens, row)
    elif row['sex'] == 'Womens':
        agregar_a_diccionario(womens, row)

# Imprimir los diccionarios para verificar
print("Hombres:", mens)
print("Mujeres:", womens)

Hombres: {'jagger eaton': [[78.0], [78.0], [78.0], [78.0], [84.23], [84.23], [82.2], [93.0], [93.0], [93.0], [93.0], [93.0], [88.33], [88.33], [88.33], [88.33], [77.17], [77.17], [89.49], [34.15], [34.15], [34.15], [34.15], [86.51], [86.51], [85.5]], 'yuro nagahara': [[75.33], [75.33], [75.33], [75.33], [62.0], [62.0], [62.0], [62.0], [62.0], [62.0], [78.68], [78.68], [78.68], [78.68], [82.0], [82.0], [82.0], [82.0], [84.43], [84.43], [88.14], [79.2], [79.2], [79.2], [71.06], [71.06], [71.06], [78.86], [78.86], [79.2], [81.99], [62.6], [62.6], [62.6], [81.1], [81.1]], 'augusto akio': [[74.0], [74.0], [74.0], [74.0], [85.62], [85.62], [88.76], [92.0], [92.0], [92.0], [92.0], [92.0], [84.5], [84.5], [84.5], [84.5], [78.46], [78.46], [87.59], [77.86], [77.86], [77.86], [77.86], [84.48]], 'tate carew': [[72.66], [72.66], [72.66], [72.66], [77.9], [77.9], [81.33], [85.13], [85.13], [85.13], [85.13], [85.13], [91.34], [91.34], [91.34], [91.34], [75.95], [75.95], [89.32], [83.19], [83.19], [8

In [74]:
def evaluate_point(x,kde):
    punto_eval = np.array([[x]])
    log_densidad_punto = kde.score_samples(punto_eval)
    return np.exp(log_densidad_punto) 

In [75]:
def global_max(kde):
    
    # Puntos donde queremos evaluar la densidad
    X_d = np.linspace(0, 100,1000)[:, np.newaxis]

    # Evaluar la densidad en los puntos X_d
    log_densidad = kde.score_samples(X_d)
    densidad = np.exp(log_densidad)
    return np.max(densidad)

In [76]:
def kde_function(data):
    # Datos de ejemplo
    X = np.array(data)
    # Instanciar el estimador de densidad de kernel con kernel de Epanechnikov
    kde = KernelDensity(kernel='epanechnikov', bandwidth=35).fit(X)

    return kde

In [77]:
def auxiliar_v_a():
    return 100 * np.random.uniform(0,1)

In [78]:
def lista_densidad(kde):
    # Puntos donde queremos evaluar la densidad
    X_d = np.linspace(0, 100,1000)[:, np.newaxis]

    # Evaluar la densidad en los puntos X_d
    log_densidad = kde.score_samples(X_d)
    return np.exp(log_densidad)

In [79]:
# Datos de ejemplo
X = np.array(mens["jagger eaton"])

# Instanciar el estimador de densidad de kernel con kernel de Epanechnikov
kde = KernelDensity(kernel='epanechnikov', bandwidth=35).fit(X)

# Puntos donde queremos evaluar la densidad
X_d = np.linspace(0, 100,1000)[:, np.newaxis]

# Evaluar la densidad en los puntos X_d
log_densidad = kde.score_samples(X_d)
densidad = np.exp(log_densidad)

In [80]:
# Generar muestras usando el método de aceptación-rechazo
def aceptar_rechazar(kde):

    while True:
        # Generar X ~ U(0, 1/c)
        X = auxiliar_v_a()
        # Generar U ~ U(0, 1)
        U = np.random.uniform(0, 1)
        # Verificar condición de aceptación
        if U <= evaluate_point(X,kde)/ global_max(kde):  # g(x) = 1 para U(0, 1)
            return X 

In [81]:
class Atleta:
    def __init__(self, nombre, datos):
        self.nombre = nombre
        self.datos = datos
        self.kde = kde_function(datos)

In [82]:
Mens=[]
Womens=[]
for key,value in mens.items():
    Mens.append(Atleta(key,value))
for key,value in womens.items():
    Womens.append(Atleta(key,value))